<a href="https://colab.research.google.com/github/brhie/ML-Algorithims-from-Scratch/blob/main/Decision_Tree_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
X_train = np.array([
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 0],
    [0, 1, 1],
    [1, 1, 0],
    [1, 0, 1],
    [1, 1, 1]
])
y_train = np.array([1, 0, 0, 1, 1, 0, 0, 1])
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (8, 3)
y_train shape: (8,)


In [ ]:
def compute_entropy(y):
    entropy = 0.

    if len(y) != 0:
        p_1 = len(y[y == 1]) / len(y)
        if p_1 != 0 and p_1 != 1:
            entropy = - p_1 * np.log2(p_1) - (1 - p_1) * np.log2(1 - p_1)

    return entropy

def split_dataset(X, node_indices, feature):
    left_indices = []
    right_indices = []

    for i in node_indices:
        if X[i, feature] == 1:
            left_indices.append(i)
        elif X[i, feature] == 0:
            right_indices.append(i)
        else:
            pass

    return left_indices, right_indices


def compute_information_gain(X, y, node_indices, feature):
    left_indices, right_indices = split_dataset(X, node_indices, feature)

    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]

    information_gain = 0
    node_entropy = compute_entropy(y_node)
    left_entropy = compute_entropy(y_left)
    right_entropy = compute_entropy(y_right)

    w_left = len(left_indices) / len(node_indices)
    w_right = len(right_indices) / len(node_indices)

    information_gain = node_entropy - (w_left * left_entropy + w_right * right_entropy)
    return information_gain

def get_best_split(X, y, node_indices):
    num_features = X.shape[1]
    best_feature = -1
    best_info_gain = 0

    for feature in range(num_features):
        info_gain = compute_information_gain(X, y, node_indices, feature=feature)
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = feature

    return best_feature

tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return

    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices)

    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))

    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))

    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [ ]:
root_indices = [i for i in range(len(X_train))]
build_tree_recursive(X_train, y_train, root_indices, branch_name="node", max_depth=2, current_depth=0)

 Depth 0, node: Split on feature: -1
- Depth 1, Left: Split on feature: 1
  -- Left leaf node with indices [4, 7]
  -- Right leaf node with indices [1, 6]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [2, 5]
  -- Right leaf node with indices [0, 3]


In [ ]:
tree

[([1, 4, 6, 7], [0, 2, 3, 5], -1),
 ([4, 7], [1, 6], 1),
 ([4, 7], [], -1),
 ([4, 7], [], -1),
 ([1, 4, 6, 7], [0, 2, 3, 5], -1),
 ([4, 7], [1, 6], 1),
 ([2, 5], [0, 3], 1)]